# RAG Using LangChain & SingleStore

### Install libraries & dependencies

In [31]:
!pip install langchain --quiet
!pip install --upgrade openai==0.28.1 --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install singlestoredb --quiet
!pip install tiktoken --quiet
!pip install --upgrade unstructured==0.10.14 --quiet
!pip install -qU pypdf langchain_community

### Import the libraries

In [38]:
from langchain.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os

### Load your custom document

In [40]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "https://unctad.org/system/files/official-document/wesp2023_en.pdf"
loader = PyPDFLoader(file_path)

data = loader.load()

### Split the document into chunks

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print(f"You have {len(data)} document(s) in your data")
print(f"There are {len(data[0].page_content)} characters in your document")

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

print(f"You have {len(texts)} pages")

You have 178 document(s) in your data
There are 44 characters in your document
You have 379 pages


### Useing OpenAI API to generate embeddings for the document chunks

In [47]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

OpenAI API Key:  ········


### Let's store our document embeddings into SingleStore database

In [48]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

#from langchain.vectorstores.singlestoredb as s2
from langchain.vectorstores import SingleStoreDB
#from langchain.vectorstores.utils import DistanceStrategy

#s2.ORDERING_DIRECTIVE["DOT_PRODUCT"] = s2.ORDERING_DIRECTIVE[DistanceStrategy.DOT_PRODUCT]

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "pdf_un",
    #distance_strategy = "DOT_PRODUCT"
)

/tmp/ipykernel_51/1637386413.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


### Let us check the text chunks and associated embeddings stored inside our database

In [50]:
%%sql
select * from pdf_un limit 1;

1 rows affected.

+------------------+-------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Ask a query against your custom data (the pdf that you loaded) using just similarity search to retrieve the top k closest content

In [51]:
query = "What India's GDP growth is projected to be?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

99cHAPtER  iii . REGion Al DEvEloPmEnt S AnD outlooK
South Asia: A challenging road 
ahead amid global headwinds
 ● South Asia’s outlook has deteriorated 
amid challenging domestic and 
global conditions.
 ● Rising global food and energy prices are 
intensifying pressure on food security 
and undermining progress on the SDGs.
 ● The economic impact of the conflict 
in Ukraine is exacerbating existing 
vulnerabilities across the region.
The outlook for South Asia has deteriorated and 
is subject to multiple downside risks amid global 
monetary tightening, fiscal vulnerabilities, rising 
inflation and extreme weather events. Regional 
GDP growth is expected to slow to 4.8 per cent 
in 2023 from an estimated 5.6 per cent expansion 
in 2022. Overall, weaker global demand, tighter 
monetary policy, additional supply disruptions, 
further escalation in commodity prices and 
the emergence of new COVID-19 variants pose 
significant risks in 2023.
India’s GDP growth rate is projected to moderat

### Here is the augmented response to the user query

In [52]:
import openai

prompt = f"The user asked: {query}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

India's GDP growth rate is projected to moderate to 5.8 per cent in 2023 from an estimated 6.4 per cent in 2022.


### Let’s test when knowledge base (custom documents like pdf) is not provided

In [53]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.8)

/tmp/ipykernel_51/2780666782.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.8)


In [54]:
llm.predict("What India's GDP growth is projected to be in 2024?")

/tmp/ipykernel_51/360469251.py:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("What India's GDP growth is projected to be in 2024?")


"\n\nAccording to the International Monetary Fund (IMF), India's GDP growth is projected to be 6.3% in 2024."

In [66]:
query = "SriLanka sought financial assistance from whom?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

As the United States Federal Reserve raised its 
policy rate and international investors reduced 
their exposures to developing markets in 2022, 
South Asian currencies weakened significantly 
against the dollar. In response, central banks in 
the region accelerated their interest rate hikes 
and intervened strongly to prevent further 
currency depreciation, particularly during the 
second half of the year. Fiscal and balance-of-
payments financing needs were exacerbated 
across the region.
Existing high levels of sovereign debt and 
unsustainable debt-servicing burdens 
prompted several South Asian countries to seek 
multilateral financial support in the second 
half of 2022 (figure III.15). After defaulting on 
its sovereign debt in April, Sri Lanka reached a 
staff-level agreement with the IMF under the 
Extended Fund Facility in early September. The 
IMF programme is expected to help boost tax 
revenues and reduce fiscal deficits in coming 
years (IMF, 2022k). Pakistan and Banglade

In [67]:
import openai

prompt = f"The user asked: {query}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

Sri Lanka sought financial assistance from the International Monetary Fund (IMF) after defaulting on its sovereign debt in April. In early September, Sri Lanka reached a staff-level agreement with the IMF under the Extended Fund Facility. The IMF programme is expected to help boost Sri Lanka's tax revenues and reduce fiscal deficits in the coming years.
